Reference paper: stability is stable 
Experiment: vary the value of rho and see the minimum sample size required for replicability

func getConvergenceSampleNum(hyperparams)
    For sample_num range(min_subsets_size, max_subsets_size, step):
        For range(repeat_num):
            random draw a subset of the dataset, whose size = sample_num
            Use algorithm 10 to get a model
        Check whether the models we get are 'replicable' to each other, accoding to the hyperparams. (see definition in page 14 of the paper)
        if converged, return current sample_num

func experiment(): 
# vary the rho and see minumum sample size required for replicability. 
# To make it simple, we can fix the value of other hyperparamers, such as alpha and beta.
    for rho in range(min_rho, max_rho, step):
        sample_num = getConvergenceSampleNum(rho)
        theoretical_sample_num = getTheoreticalSampleNum(rho) # compute according to algorithm 10 in the paper
        print("Hyperparams: ", hyperparams, "Sample size: ", sample_num)
    plot(hyperparams, theoretical_sample_num) # draw the curve, where x-axis is the hyperparams and y-axis is the sample size
    plot(hyperparams, sample_num) # draw the curve, where x-axis is the hyperparams and y-axis is the sample size

In [1]:
import pandas as pd
import numpy as np
import config
import Algorithm10 as a10
from sklearn.tree import export_text

##### Config Variables #####
np = <module 'numpy' from '/opt/anaconda3/envs/myenv/lib/python3.13/site-packages/numpy/__init__.py'>
dataset_path = ./dataset/Invistico_Airline.csv
model_path = ./models/
max_depth = 3
random_seed = 42
selected_features = ['Class', 'Seat comfort', 'Food and drink', 'Cleanliness', 'satisfaction']
rho = 0.3
alpha = 0.1
beta = 0.1
num_H = 10
m_up_bound = 81365.1085764509
tau_up_bound = 0.013028834457097554
tau = 0.0013028834457097555
m = 100
############################


In [2]:
data = pd.read_csv('dataset/Invistico_Airline.csv')
data.head()

,satisfaction,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,...,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,satisfied,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,2,...,2,3,3,0,3,5,3,2,0,0.0
1,satisfied,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,3,...,2,3,4,4,4,2,3,2,310,305.0
2,satisfied,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,3,...,2,2,3,3,4,4,4,2,0,0.0
3,satisfied,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,3,...,3,1,1,0,1,4,1,3,0,0.0
4,satisfied,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,3,...,4,2,2,0,2,4,2,5,0,0.0


In [3]:
data.shape

(129880, 22)

In [ ]:
# function to check if two decision trees are equal
def are_trees_equal(tree1, tree2):
    # Check that both trees are fitted
    if not hasattr(tree1, 'tree_') or not hasattr(tree2, 'tree_'):
        raise ValueError("Both trees must be fitted before comparison.")

    # Compare parameters
    if tree1.get_params() != tree2.get_params():
        return False

    t1 = tree1.tree_
    t2 = tree2.tree_

    # Compare structure and splitting rules
    attributes_to_check = [
        'children_left', 'children_right',
        'feature', 'threshold',
        'impurity', 'n_node_samples', 'weighted_n_node_samples',
        'value'
    ]

    for attr in attributes_to_check:
        if not np.array_equal(getattr(t1, attr), getattr(t2, attr)):
            return False

    return True


In [5]:
def getConvergenceSampleNum(min_subset_size, max_subset_size, repeat_num, rho, sample_size_step=1):
    sample_size_replicablity_dict = {}
    
    X, y= a10.load_full_dataset(config.dataset_path, random_state=config.random_seed)
    for sample_size in range(min_subset_size, max_subset_size + 1, sample_size_step):
        #get dataset of size sample_size by sampling from the original dataset
        replicable_tree_list = []
        H = a10.build_candidate_trees(X, y,sample_size, max_depth=config.max_depth, num_trees=config.num_H, random_state=config.random_seed)    
        for i in range(repeat_num):
            print(f"sample size: {sample_size}, repeat: {i}")
            res_trees = a10.replicable_learner(X, y, H, sample_size, random_seed=config.random_seed+i)
            print(f"number of res_trees: {len(res_trees)}")
            # check whether the candidate trees below v are same to each other
            for a in range(len(res_trees)):
                for b in range(a + 1, len(res_trees)):
                    if are_trees_equal(res_trees[a], res_trees[b]):
                        print(f"tree {a} and tree {b} are the same")
                    else:
                        print(f"tree {a} and tree {b} are different")
                        print("tree a",export_text(res_trees[a]),"tree b",export_text(res_trees[b]))
                
            tree = res_trees[0]
            replicable_tree_list.append(tree)
            
        #check the probability if the trees in the replicable_tree_list are the same
        same_tree_count = 0
        for i in range(len(replicable_tree_list)):
            for j in range(i + 1, len(replicable_tree_list)):
                
                if are_trees_equal(replicable_tree_list[i], replicable_tree_list[j]):
                    # print(f"tree {i} and tree {j} are the same")
                    same_tree_count += 1
        prob = same_tree_count / (repeat_num * (repeat_num - 1) / 2)
        sample_size_replicablity_dict[sample_size] = prob
        if prob >= 1-rho:
            print(f"replicable at sample size: {sample_size}, prob: {prob}")
            return sample_size 
        
    # return sample_size_replicablity_dict
    print(f"not replicable at sample size between {min_subset_size} and {max_subset_size}, prob: {prob}")
    return -1

            
        
    

In [6]:
print("theoretical sample size: ", config.m_up_bound)
ans_dict = getConvergenceSampleNum(min_subset_size=100, max_subset_size=200, repeat_num=10, rho=config.rho, sample_size_step=100)

theoretical sample size:  81365.1085764509


100%|██████████| 10/10 [00:00<00:00, 150.18it/s]

sample size: 100, repeat: 0
OPT error 0.21999999999999997 errors dict_values([0.27, 0.31000000000000005, 0.28, 0.21999999999999997, 0.24, 0.35, 0.30000000000000004, 0.24, 0.32999999999999996, 0.29000000000000004])
k: 18 v max candidates: 0.2430444513602209 v min candidates: 0.22089543278315507
[np.float64(0.22089543278315507), np.float64(0.22219831622886482), np.float64(0.22350119967457457), np.float64(0.22480408312028433), np.float64(0.22610696656599408), np.float64(0.22740985001170383), np.float64(0.22871273345741358), np.float64(0.23001561690312336), np.float64(0.2313185003488331), np.float64(0.23262138379454286), np.float64(0.23392426724025261), np.float64(0.23522715068596237), np.float64(0.23653003413167212), np.float64(0.23783291757738187), np.float64(0.23913580102309162), np.float64(0.2404386844688014), np.float64(0.24174156791451115), np.float64(0.2430444513602209)]
v: 0.22480408312028433
number of res_trees: 1
sample size: 100, repeat: 1
OPT error 0.22999999999999998 errors di

OPT error 0.19999999999999996 errors dict_values([0.31999999999999995, 0.35, 0.26, 0.20999999999999996, 0.21999999999999997, 0.29000000000000004, 0.26, 0.19999999999999996, 0.28, 0.29000000000000004])
k: 18 v max candidates: 0.22344474366418254 v min candidates: 0.2012957250871167
[np.float64(0.2012957250871167), np.float64(0.20259860853282646), np.float64(0.20390149197853621), np.float64(0.20520437542424597), np.float64(0.20650725886995572), np.float64(0.20781014231566547), np.float64(0.20911302576137522), np.float64(0.210415909207085), np.float64(0.21171879265279475), np.float64(0.2130216760985045), np.float64(0.21432455954421425), np.float64(0.215627442989924), np.float64(0.21693032643563376), np.float64(0.2182332098813435), np.float64(0.21953609332705326), np.float64(0.22083897677276304), np.float64(0.2221418602184728), np.float64(0.22344474366418254)]
v: 0.21171879265279475
number of res_trees: 2
tree 0 and tree 1 are different
tree a |--- feature_1 <= 3.50
|   |--- feature_0 <= 0

100%|██████████| 10/10 [00:00<00:00, 177.27it/s]


sample size: 200, repeat: 0
OPT error 0.22499999999999998 errors dict_values([0.24, 0.30000000000000004, 0.31000000000000005, 0.26, 0.235, 0.22499999999999998, 0.24, 0.22499999999999998, 0.235, 0.29500000000000004])
k: 18 v max candidates: 0.2480444513602209 v min candidates: 0.22589543278315508
[np.float64(0.22589543278315508), np.float64(0.22719831622886483), np.float64(0.22850119967457458), np.float64(0.22980408312028433), np.float64(0.23110696656599408), np.float64(0.23240985001170383), np.float64(0.23371273345741359), np.float64(0.23501561690312336), np.float64(0.23631850034883312), np.float64(0.23762138379454287), np.float64(0.23892426724025262), np.float64(0.24022715068596237), np.float64(0.24153003413167212), np.float64(0.24283291757738187), np.float64(0.24413580102309163), np.float64(0.2454386844688014), np.float64(0.24674156791451116), np.float64(0.2480444513602209)]
v: 0.22980408312028433
number of res_trees: 2
tree 0 and tree 1 are different
tree a |--- feature_1 <= 3.50
| 

In [7]:
print(config.rho)

0.3
